The below performs FFT on 4 channels of 8K samples

In [2]:
import socket,struct,time
import numpy as np
import select

from bokeh.client.session import ClientSession
from bokeh.client import push_session
from bokeh.plotting import figure, curdoc
from bokeh.io import show, vform
from bokeh.models.widgets import RadioGroup, RadioButtonGroup

###### Sampling definitions ######
# No of samples per channel
N  = 8192
# sample rate
FS = 20000
# Time between each block can be retrieved, in [mS]
T_COMM = (N*1000)/FS
# 4 sample channels + info data
MSGLEN = 65542  #((8*8192) +4)

###### Comm settings ######
SERVER_ADDRESS = "192.168.1.102"
SERVER_PORT = 7

###### Vibration analyzer / logger class ######
class VibrationLogger(object):
    '''
    classdocs
    '''
   # constuctor
    def __init__(self ):
        # 4 numpy arrays to hold data        
        self.vibData2 = np.arange(N)
                                  
        #[4000][][][]                          
        self.vibData = np.array([[np.arange(N)],[np.arange(N)],[np.arange(N)],[np.arange(N)]])
        self.fftData = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
        self.fftData1 = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
        self.fftData2 = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
        self.fftData3 = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
        self.fftDataDisp = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
        
        self.tmp1 = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])
        self.tmp2 = np.array([[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)],[np.zeros((N/2)+1)]])



        
        self.currentRpmValue = 3000;
        self.sampleFlags = 0
        
        self.channelsActive = [1,0,0,0]
        self.channelSelected = 0
        self.doFFT = 1;

        # get Hanning window constants
        self.FFTwindow = np.hanning(N)
        # Create arrays for plotting X and Y axis
        self.x = np.linspace(0, FS, N)
        self.t1 = 0
        self.t2 = 0
        
        # availible TOOLs 
        TOOLS="resize,crosshair,pan,wheel_zoom,box_zoom,reset,tap,box_select,lasso_select,hover"
        self.plot_kw = dict(tools=TOOLS, h_symmetry=False, v_symmetry=False,  outline_line_color='#595959',)
        
        # plot headings etc
        self.p = figure(webgl=True,title="FFT test", plot_width=1200, plot_height=600, y_axis_type="log", **self.plot_kw)
        self.channelColors = ["red","blue","green","magenta"]
        self.r = self.p.line(x=[], y=[], color=self.channelColors[1], line_width=1)
        self.p.ygrid.minor_grid_line_color = 'navy'
        self.p.ygrid.minor_grid_line_alpha = 0.1
        self.p.xaxis.axis_label = "Frequency [Hz]"
        self.p.yaxis.axis_label = "Vibration [mG]"

        # attach "ds" to data feed
        self.ds = self.r.data_source        
        self.rpmMarker = self.p.triangle(x=[], y=1, size=25, color="red", alpha=0.8)
        self.rpmds = self.rpmMarker.data_source
        
        # channel selctor buttons
        self.channelSelectorButtons = RadioGroup(labels=["Channel 1", "Channel 2", "Channel 3", "Channel 4"], active=0)
        self.channelSelectorButtons.on_click(self.chanSelUpdate)
        show(vform(self.channelSelectorButtons))
        
        self.isTimeFFTAnalysis = RadioButtonGroup(labels=["Time", "FFT"], active=1)
        self.isTimeFFTAnalysis.on_click(self.timeFFTUpdate)
        show(vform(self.isTimeFFTAnalysis))

        # open a session to keep our local document in sync with server
        self.session = push_session(curdoc())
        curdoc().add_periodic_callback(self.update, T_COMM) #800

    def connectTcp(self):
        self.tcpConn = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # set no delay to avoid 200 ms delayed ACK from server - TODO verify !!
        #self.tcpConn.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
        self.tcpConn.connect((SERVER_ADDRESS, SERVER_PORT))
        self.tcpConn.setblocking(0) #added     
        print ('connect to NODE %s:%d' % (SERVER_ADDRESS, SERVER_PORT))
     
    def timeFFTUpdate(self, new):
        if(new == 0):
            self.doFFT = 0;
        else:
            self.doFFT = 1;
        
    def chanSelUpdate(self, new):
        self.channelSelected = new
        print 'Channel ' + str(new) + ' selected.'
        self.p.title = "Data from channel :" + str(self.channelSelected+1)

    def doFFTs(self):        
        #update delay line
        np.copyto(self.fftData3, self.fftData2)
        np.copyto(self.fftData2, self.fftData1)
        np.copyto(self.fftData1, self.fftData)

        #self.fftData3 = self.fftData2
        #self.fftData2 = self.fftData1
        #self.fftData1 = self.fftData

        # perform FFT on the windowed samples
        for n in range(4):
            self.fftData[n] = np.fft.rfft(self.vibData[n] * self.FFTwindow ) /N 
            self.fftData[n,0,0]=0 #zero out DC    
            
        # a simple average
        # ADD ONLY TAKES TWO ARGUMENTS
        self.tmp1= np.add(self.fftData3, self.fftData2)
        self.tmp2= np.add(self.fftData1, self.fftData)
        fftDataDisp = np.add(self.tmp1, self.tmp2)
        self.fftData.mean()
        #(self.fftData3+self.fftData2+self.fftData1+self.fftData)/4
  
    def requestVibData(self):       
        # protocol
        message = 'GET_SAMPLES\x00'
        #request samples
        self.tcpConn.sendall(message)
        #print 'requesting data'
    
    def update(self):
        self.requestVibData()
        readers,_,_ = select.select([self.tcpConn],[],[])
        if readers:
            # struct containing 4 channels RPM and Status flags
            s1 = struct.Struct('8192H 8192H 8192H 8192H H H H')
            #rawData = self.recv_size(MSGLEN)
            rawData = self.tcpConn.recv(MSGLEN)
            
            if(len(rawData) == MSGLEN):
                sortedData = np.asarray(s1.unpack(rawData)[:-3])
                self.vibData = sortedData.reshape(4,8192)
                
                sortedData = np.asarray(s1.unpack(rawData)[-3:])
                self.currentRpmValue    = sortedData[0]
                self.sampleFlags        = sortedData[1]
                self.msSinceLastFrame   = sortedData[2]
                # we got data, compute FFT's
                self.doFFTs()
                # verify time between frames
                self.t1 = self.t2
                self.t2 = time.clock()
                print 't1:',self.t1,'t2:',self.t2, 'delta:',self.t2-self.t1 #(self.t2-self.t1)
                
                
            self.ds.data["x"] = self.x
            #self.ds.data["y"] = self.fftData3[self.channelSelected,0]     #vib data is [4:8K] -fftData
            self.ds.data["y"] = self.fftDataDisp[self.channelSelected,0]     #vib data is [4:8K] -fftData
            


In [9]:
x= np.zeros(N*4)
x=x.reshape((N,4))
x.shape
m = x.mean(1)
m.shape

(8192,)

Here we create an instance of the vibration logger, and starts logging

In [2]:
print 'sample settings: T_COMM',T_COMM,'[ms]'
vibLog = VibrationLogger()
vibLog.connectTcp()
#    vibLog.addRpmLines()
vibLog.session.show() # open the document in a browser       
vibLog.session.loop_until_closed() # run forever


sample settings: T_COMM 409 [ms]
connect to NODE 192.168.1.102:7
t1: 0 t2: 1.893716 delta: 1.893716
t1: 1.893716 t2: 1.993934 delta: 0.100218
t1: 1.993934 t2: 2.050813 delta: 0.056879
t1: 2.050813 t2: 2.107162 delta: 0.056349
t1: 2.107162 t2: 2.183238 delta: 0.076076
t1: 2.183238 t2: 2.258493 delta: 0.075255
t1: 2.258493 t2: 2.327889 delta: 0.069396
t1: 2.327889 t2: 2.399038 delta: 0.071149
t1: 2.399038 t2: 2.467022 delta: 0.067984
t1: 2.467022 t2: 2.535706 delta: 0.068684
t1: 2.535706 t2: 2.601304 delta: 0.065598
t1: 2.601304 t2: 2.669384 delta: 0.06808
t1: 2.669384 t2: 2.740006 delta: 0.070622
t1: 2.740006 t2: 2.811029 delta: 0.071023
t1: 2.811029 t2: 2.880371 delta: 0.069342
t1: 2.880371 t2: 2.951375 delta: 0.071004
t1: 2.951375 t2: 3.026163 delta: 0.074788
t1: 3.026163 t2: 3.094925 delta: 0.068762
t1: 3.094925 t2: 3.163031 delta: 0.068106
t1: 3.163031 t2: 3.230912 delta: 0.067881
t1: 3.230912 t2: 3.301545 delta: 0.070633
t1: 3.301545 t2: 3.372624 delta: 0.071079
t1: 3.372624 t2: 3.

/Library/Python/2.7/site-packages/ipykernel/__main__.py:125: ComplexWarning: Casting complex values to real discards the imaginary part
